In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [2]:
!pip install ortools

In [3]:
import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp

# 1) Read and Preprocessing Data

In [4]:
path = '/content/gdrive/MyDrive/fantasy_football_data.csv'

In [5]:
data = pd.read_csv(path) # + 'fantasy_football_data.csv')

In [6]:
positions = data.Position.unique()

In [7]:
#Number of players in the starting lineup team
num_lineup_players = {'QB':1,'WR':3,'RB':2,'TE':1,'K':1,'DEF':1}

In [8]:
#Number of potential players playing each positions in the raw dataset
num_position_choice = data.groupby('Position').count()['Name'].to_dict()

In [9]:
#Give players ids to differentiate them in case of the same names
list_id = [[[pos + str(j) for j in range(1,i+1)] for i in [num_position_choice[pos]]] for pos in positions]

In [10]:
# #Function flattening 3D nested list
def flatten_nested_list(lst):
    first_list = []
    for sublist in lst:
        for item in sublist:
            first_list.append(item)
    second_list = []
    for sublist in first_list:
        for item in sublist:
            second_list.append(item)
    return second_list

In [11]:
data['id'] = flatten_nested_list(list_id)

In [12]:
data.head()

,Name,Position,Team,ProjectedPoints,Price,id
0,Aaron Rodgers,QB,GBP,333.5,51.2,QB1
1,Tom Brady,QB,NEP,312.1,38.3,QB2
2,Drew Brees,QB,NOS,306.2,36.3,QB3
3,Matt Ryan,QB,ATL,294.3,29.2,QB4
4,Russell Wilson,QB,SEA,287.9,25.8,QB5


In [13]:
#Discount factor for the expected points of players on the Bench
discount_factor = {pos:{dep:(1/2)**dep for dep in range(1,7)} for pos in positions}

In [14]:
#Get the prices of players from the raw data
cost_per_player = data.groupby('Position')[['Price','id']].apply(lambda x: x.set_index('id').to_dict(orient='index')).to_dict()

In [15]:
#Get the projected points of players from the raw data
point_per_player = data.groupby('Position')[['ProjectedPoints','id']].apply(lambda x: x.set_index('id').to_dict(orient='index')).to_dict()

# 2) Solver and Variables

In [16]:
#Create solver
solver = pywraplp.Solver.CreateSolver('SCIP')

In [17]:
#Create variables representing for who will be on the Bench
bench_player = {pos:{dep:{player: solver.IntVar(0,1, str(player) + '_B_' + str(dep)) for player in data.loc[data['Position'] == pos]['id']} for dep in range(1,7)} for pos in positions}

In [18]:
#Create variables representing for who will be in the starting lineup team
lineup_player = {pos:{dep:{player: solver.IntVar(0,1, str(player) + '_L_' + str(dep)) for player in data.loc[data['Position'] == pos]['id']} for dep in range(1,7)} for pos in positions}

# 3) Objective function

In [19]:
#Create objective function which maximizing total points of players in the lineup team and on the Bench
Expected_points_lineup_team = solver.Sum([solver.Sum([solver.Sum([lineup_player[pos][dep][player]*point_per_player[pos][player]['ProjectedPoints'] for player in data.loc[data['Position'] == pos]['id']]) for dep in range(1,7)]) for pos in positions])
Expected_points_bench_team = solver.Sum([solver.Sum([solver.Sum([bench_player[pos][dep][player]*point_per_player[pos][player]['ProjectedPoints']*discount_factor[pos][dep] for player in data.loc[data['Position'] == pos]['id']]) for dep in range(1,7)]) for pos in positions])

solver.Maximize(Expected_points_lineup_team + Expected_points_bench_team)

# 4) Constraints

## 4.1) Minimum lineup player

In [21]:
#A roster has 1 QB slot, 3 WR slots, 2 RB slots, 1 TE slot, 1 K slot, 1 DEF slot
for pos in positions:
    solver.Add(solver.Sum([solver.Sum([lineup_player[pos][dep][player] for player in data.loc[data['Position'] == pos]['id']]) for dep in range(1,7)]) == num_lineup_players[pos])

In [22]:
#No players in the starting lineup team can be double counted
for pos in positions:
    for player in data.loc[data['Position'] == pos]['id']:
        solver.Add(solver.Sum([lineup_player[pos][dep][player] for dep in range(1,7)]) <= 1)

## 4.2) Maximum bench player

In [23]:
#6 Bench slots - A Bench slot can contain any player type
for pos in positions:
    solver.Add(solver.Sum([solver.Sum([bench_player[pos][dep][player]for player in data.loc[data['Position'] == pos]['id']]) for dep in range(1,7)]) <= 6)

In [24]:
#No players on the Bench can be double counted
for pos in positions:
    for player in data.loc[data['Position'] == pos]['id']:
        solver.Add(solver.Sum([bench_player[pos][dep][player] for dep in range(1,7)]) <= 1)

In [25]:
#Maximum number of players of the Bench is 6
solver.Add(solver.Sum([solver.Sum([solver.Sum([bench_player[pos][dep][player] for player in data.loc[data['Position'] == pos]['id']]) for dep in range(1,7)]) for pos in positions]) == 6)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7f0c04ea6450> >

## 4.3) Singularity constraint

In [26]:
#A player on your bench team can only be assigned to one slot
for pos in positions:
    for dep in range(1,7):
        solver.Add(solver.Sum([bench_player[pos][dep][player] for player in data.loc[data['Position'] == pos]['id']]) <= 1)

In [27]:
#A player on your lineup team can only be assigned to one slot
for pos in positions:
    for player in data.loc[data['Position'] == pos]['id']:
        solver.Add(solver.Sum([solver.Sum([lineup_player[pos][dep][player],bench_player[pos][dep][player]]) for dep in range(1,7)]) <= 1)

In [28]:
#Players on the lineup team perform better than players on the Bench
for pos in positions:
    solver.Add(solver.Sum([lineup_player[pos][1][player]*point_per_player[pos][player]['ProjectedPoints'] for player in data.loc[data['Position'] == pos]['id']]) \
               - solver.Sum([bench_player[pos][1][player]*point_per_player[pos][player]['ProjectedPoints'] for player in data.loc[data['Position'] == pos]['id']]) >= 0)
    solver.Add(solver.Sum([lineup_player[pos][6][player]*point_per_player[pos][player]['ProjectedPoints'] for player in data.loc[data['Position'] == pos]['id']]) \
               - solver.Sum([bench_player[pos][6][player]*point_per_player[pos][player]['ProjectedPoints'] for player in data.loc[data['Position'] == pos]['id']]) >= 0)
    for dep in range(1,6):
        solver.Add(solver.Sum([lineup_player[pos][dep][player]*point_per_player[pos][player]['ProjectedPoints'] for player in data.loc[data['Position'] == pos]['id']]) \
                - solver.Sum([bench_player[pos][dep+1][player]*point_per_player[pos][player]['ProjectedPoints'] for player in data.loc[data['Position'] == pos]['id']]) >= 0)

## 4.4) Budget constraint

In [29]:
#Totla budget is less than 200
Budget_lineup = solver.Sum([solver.Sum([solver.Sum([(lineup_player[pos][dep][player])*(cost_per_player[pos][player]['Price']) for player in data.loc[data['Position'] == pos]['id']]) for dep in range(1,7)]) for pos in positions])
Budget_bench = solver.Sum([solver.Sum([solver.Sum([bench_player[pos][dep][player] * cost_per_player[pos][player]['Price'] for player in data.loc[data['Position'] == pos]['id']]) for dep in range(1,7)]) for pos in positions])
solver.Add((Budget_lineup + Budget_bench) <= 200)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7f0bfd3c6a50> >

## 4.5) Continuity constraint

In [30]:
#Select Bench players in order to rank them according to their projected points
for pos in positions:
    for dep in range(1,6):
        continuity1 = solver.Sum([bench_player[pos][dep][player] for player in data.loc[data['Position'] == pos]['id']]) \
                    - solver.Sum([bench_player[pos][dep+1][player] for player in data.loc[data['Position'] == pos]['id']])
        solver.Add(continuity1 >= 0)

In [31]:
#Select Lineup players in order to make sure that the depth in the lineup team will be satisfied
for pos in positions:
    for dep in range(1,6):
        continuity2 = solver.Sum([lineup_player[pos][dep][player] for player in data.loc[data['Position'] == pos]['id']]) \
                    - solver.Sum([lineup_player[pos][dep+1][player] for player in data.loc[data['Position'] == pos]['id']])
        solver.Add(continuity2 >= 0)

## 4.6) At most constraint

In [32]:
#At most 2 QBs
QB_lineup = solver.Sum([solver.Sum([lineup_player['QB'][dep][player] for player in data.loc[data['Position'] == 'QB']['id']]) for dep in range(1,7)])
QB_Bench = solver.Sum([solver.Sum([bench_player['QB'][dep][player] for player in data.loc[data['Position'] == 'QB']['id']]) for dep in range(1,7)])
solver.Add(QB_lineup+QB_Bench <= 2)        

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7f0bfdb2d300> >

In [33]:
#At most 2 TEs
TE_lineup = solver.Sum([solver.Sum([lineup_player['TE'][dep][player] for player in data.loc[data['Position'] == 'TE']['id']]) for dep in range(1,7)])
TE_Bench = solver.Sum([solver.Sum([bench_player['TE'][dep][player] for player in data.loc[data['Position'] == 'TE']['id']]) for dep in range(1,7)])
solver.Add(TE_lineup+TE_Bench <= 2)        

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7f0bfd186360> >

In [34]:
#At most 1 K:
K_lineup = solver.Sum([solver.Sum([lineup_player['K'][dep][player] for player in data.loc[data['Position'] == 'K']['id']]) for dep in range(1,7)])
K_Bench = solver.Sum([solver.Sum([bench_player['K'][dep][player] for player in data.loc[data['Position'] == 'K']['id']]) for dep in range(1,7)])
solver.Add(K_lineup+K_Bench <= 1)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7f0bfd186480> >

In [35]:
#At most 1 DEF:
DEF_lineup = solver.Sum([solver.Sum([lineup_player['DEF'][dep][player] for player in data.loc[data['Position'] == 'DEF']['id']]) for dep in range(1,7)])
DEF_Bench = solver.Sum([solver.Sum([bench_player['DEF'][dep][player] for player in data.loc[data['Position'] == 'DEF']['id']]) for dep in range(1,7)])
solver.Add(DEF_lineup+DEF_Bench <= 1)        

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7f0bfd602ed0> >

# 5) Solutions

In [37]:
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    print(sum([sum([sum([lineup_player[pos][dep][player].solution_value() for player in data.loc[data['Position'] == pos]['id']]) for dep in range(1,7)]) for pos in positions]))
    print(sum([sum([sum([bench_player[pos][dep][player].solution_value() for player in data.loc[data['Position'] == pos]['id']]) for dep in range(1,7)]) for pos in positions]))

Solution:
Objective value = 1754.25
9.0
6.0


In [38]:
lineup_result = [[[lineup_player[pos][dep][player].solution_value() for player in data.loc[data['Position'] == pos]['id']] for dep in range(1,7)] for pos in positions]

In [39]:
bench_result = [[[bench_player[pos][dep][player].solution_value() for player in data.loc[data['Position'] == pos]['id']] for dep in range(1,7)] for pos in positions]

In [40]:
#Information of players in the bench team in the raw data
bench_infor = [[[bench_player[pos][dep][player] for player in data.loc[data['Position'] == pos]['id']] for dep in range(1,7)] for pos in positions]

In [41]:
#Information of players in the lineup team in the raw data
lineup_infor = [[[lineup_player[pos][dep][player] for player in data.loc[data['Position'] == pos]['id']] for dep in range(1,7)] for pos in positions]

In [42]:
#Function getting the information of players from the raw data based on the solution value information
def get_names(result,infor):
    flatten_result = flatten_nested_list(result)
    index = []
    for i, j in enumerate(flatten_result):
        if j == 1:
            index.append(i)
    flatten_infor = flatten_nested_list(infor)
    players_results = []
    for i in index:
        players_results.append(flatten_infor[i])
    return players_results

In [43]:
def list_to_dataframe(lst):
    first_list = []
    for items in lst:
        first_list.append(str(items).split('_'))
    
    dataframe1 = pd.DataFrame(first_list, columns = ['id','Rootster_slot','Depth'])
    x = []
    for i in range(len(first_list)):
        x.append(data.loc[data['id'] == first_list[i][0]])

    dataframe2 = pd.concat(x)
    final_dataframe = pd.merge(dataframe2,dataframe1, on='id')
    final_dataframe['ExpectedPoint'] = final_dataframe.apply(lambda row: row.ProjectedPoints*(0.5)**(int(row.Depth)) if row.Rootster_slot =='B' else row.ProjectedPoints,axis = 1)
    
    return final_dataframe

In [44]:
lineup_results_players = get_names(lineup_result,lineup_infor)
bench_results_players = get_names(bench_result, bench_infor)

In [47]:
#My final team
final_team = pd.concat([list_to_dataframe(lineup_results_players), list_to_dataframe(bench_results_players)], axis=0, ignore_index=True)

In [48]:
final_team

,Name,Position,Team,ProjectedPoints,Price,id,Rootster_slot,Depth,ExpectedPoint
0,Cam Newton,QB,CAR,278.8,16.6,QB7,L,1,278.80
1,Ty Montgomery,RB,GBP,140.9,14.9,RB20,L,1,140.90
2,C.J. Anderson,RB,DEN,128.8,7.9,RB27,L,1,128.80
3,Antonio Brown,WR,PIT,194.6,55.3,WR1,L,1,194.60
4,Julio Jones,WR,ATL,187.8,51.3,WR2,L,1,187.80
5,Emmanuel Sanders,WR,DEN,125.7,15.2,WR25,L,1,125.70
6,Justin Tucker,K,BAL,147.0,9.2,K1,L,1,147.00
7,Martellus Bennett,TE,GBP,95.9,6.9,TE9,L,1,95.90
8,Denver Broncos,DEF,DEN,129.0,7.6,DEF2,L,1,129.00
9,Tyrod Taylor,QB,BUF,255.8,5.1,QB17,B,1,127.90


In [49]:
print('The number of points my team is expected to generate is: ',sum(final_result['ExpectedPoint']))

The number of points my team is expected to generate is:  1754.2500000000002


In [50]:
print('The number of points my lineup team is expected to generate is: ', sum(list_to_dataframe(lineup_results_players)['ExpectedPoint']))

The number of points my lineup team is expected to generate is:  1428.5000000000002


In [51]:
print('The number of points my bench team is expected to generate is: ',sum(list_to_dataframe(bench_results_players)['ExpectedPoint']))

The number of points my bench team is expected to generate is:  325.75


In [52]:
print('The amount I spend on my team is: ',sum(final_result['Price']))

The amount I spend on my team is:  200.0


In [53]:
if len(final_result.Name.unique()) == 15 and len(final_result.id.unique()) == 15:
    print('Satisfy the condition that Any QB, WR, RB, TE, K, DEF slot can only contain a player of that type')
else:
    print('Dont satisfy the condition')

Satisfy the condition that Any QB, WR, RB, TE, K, DEF slot can only contain a player of that type
